In [1]:
from keras import applications

Using TensorFlow backend.


In [25]:
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': applications.mobilenet.relu6,'DepthwiseConv2D': applications.mobilenet.DepthwiseConv2D}):
    model_export = load_model('mobilenet_alpha_5classes')


In [26]:
model_export.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 130, 130, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 66, 66, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
__________

In [27]:
from keras.models import load_model
import tensorflow as tf
from pathlib import Path
from keras import backend as K

In [28]:
print("Input: %s" % model_export.input_names)
print("Output: %s" % model_export.output_names)

Input: ['input_1']
Output: ['dense_3']


In [29]:
sess = K.get_session()

f = 'alpha_retrained.pb'

In [30]:
g = sess.graph

In [31]:
saver = tf.train.Saver()

In [32]:
num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = 'output'+str(i)
    pred[i] = tf.identity(model_export.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

output nodes names are:  ['output0']


In [33]:
pred

[<tf.Tensor 'output0_1:0' shape=(?, 5) dtype=float32>]

In [34]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.tools.graph_transforms import TransformGraph

In [39]:

from tensorflow.python.tools import optimize_for_inference_lib

g = sess.graph

constant_graph = graph_util.convert_variables_to_constants(sess, g.as_graph_def(), pred_node_names)   
constant_graph = graph_util.remove_training_nodes(constant_graph)

INFO:tensorflow:Froze 141 variables.
Converted 141 variables to const ops.


In [40]:
transforms = ["quantize_weights", "quantize_nodes"]
transformed_graph_def = TransformGraph(constant_graph, model_export.input_names, pred_node_names, transforms)

In [41]:
optimize_for_inference_lib.ensure_graph_is_valid(
        transformed_graph_def)

In [42]:
optimized_graph = optimize_for_inference_lib.fold_batch_norms(
        constant_graph)

In [43]:
optimized_graph = optimize_for_inference_lib.optimize_for_inference(
        optimized_graph,
        model_export.input_names, # an array of the input node(s)
        pred_node_names, # an array of output nodes
tf.float32.as_datatype_enum)

AssertionError: output0 is not in graph

In [44]:
graph_io.write_graph(optimized_graph, 'dense_6', f, as_text=False)

'dense_6/alpha_retrained.pb'

In [11]:
g = sess.graph


#   'strip_unused_nodes(type=float, shape="1,299,299,3")',
transforms = [
  "quantize_weights", "quantize_nodes",
  'fold_batch_norms',
  'fold_old_batch_norms']
#  'quantize_weights']
#transforms = ['merge_duplicate_nodes']
#transforms = []
transformed_graph_def = TransformGraph(g.as_graph_def(), model_export.input_names, pred_node_names, transforms)
    
constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names)   




INFO:tensorflow:Froze 141 variables.
Converted 141 variables to const ops.


In [12]:
graph_io.write_graph(constant_graph, 'frozen', f, as_text=False)

'frozen/alpha_retrained.pb'

In [14]:
tf.train.write_graph(constant_graph, 'frozen', f + '.txt', as_text=True)

'frozen/alpha_retrained.pb.txt'

In [50]:
with tf.Session() as sess:
    model = model_export
#    model.load_weights('/srv/nn/weights/scratch-vgg19.h5')
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    graph_def = sess.graph.as_graph_def()
    tf.train.write_graph(graph_def, logdir='.',   name='model.pb', as_text=False)
    saver = tf.train.Saver()
    saver.save(sess, './model.ckpt')

In [54]:
freeze_graph.freeze_graph(input_graph='model.pb', input_checkpoint='model.ckpt', output_graph='frozen2', output_node_names='output0',
                         input_saver='',
                         input_binary=True,
                         restore_op_name='',
                         filename_tensor_name='',
                         clear_devices=True,
                         initializer_nodes=''
                         )

INFO:tensorflow:Restoring parameters from model.ckpt
INFO:tensorflow:Froze 141 variables.
Converted 141 variables to const ops.
